In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import warnings 
warnings.filterwarnings('ignore')

In [3]:
data = pd.read_csv('../data/sample_data.csv')
data.head()

,Gender,Age_less_35,JobRole,Attrition
0,Male,True,Laboratory Technician,0
1,Male,False,Sales Executive,1
2,Male,True,Sales Representative,1
3,Female,False,Healthcare Representative,0
4,Male,True,Sales Executive,0


In [11]:
p = data['Attrition'].value_counts(normalize= True).values

In [18]:
- np.sum(p * np.log2(p))

0.6554120818442417

In [28]:
def entropy(y):
    p = y.value_counts(normalize = True).values
    ent = - np.sum(p * np.log2(p))
    return ent

In [29]:
entropy(data['Attrition'])

0.6554120818442417

In [24]:
categories = data['Gender'].unique()

In [30]:
entropy(data[data['Gender'] == 'Male']['Attrition'])

0.6568540844475503

In [33]:
def calculate_weighted_entropy(feature,y):
    categories = y.unique()

    weighted_entropy = 0.0

    for category in categories:
        y_category = y[category == feature]
        ent_category = entropy(y_category)

        weighted_entropy += (len(y_category)/len(y)) * ent_category

    return weighted_entropy

In [40]:
calculate_weighted_entropy(data['Gender'],data['Attrition'])

0.0

In [36]:
def information_gain(feature,y):
    
    parent_entropy = entropy(y)
    child_entropy = calculate_weighted_entropy(feature,y)

    ig = parent_entropy - child_entropy

    return ig


In [39]:
for f in data.columns[:-1]:
    print(f"Feature ({f}): {information_gain(data[f], data['Attrition']).round(5)}")

Feature (Gender): 0.65541
Feature (Age_less_35): 0.02146
Feature (JobRole): 0.65541


In [42]:
information_gain(data['JobRole'], data['Attrition'])

0.6554120818442417